In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, RandomizedSearchCV

nltk.download('punkt_tab')

# Mount Google Drive
drive.mount('/content/drive')

# Load GloVe vectors
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings

# Generate sentence embeddings
def generate_sentence_embeddings(sentences, embeddings, embedding_dim=300):
    sentence_vectors = []
    for sentence in sentences:
        tokens = word_tokenize(str(sentence).lower())
        word_vectors = [embeddings[word] for word in tokens if word in embeddings]
        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            sentence_vector = np.zeros(embedding_dim)
        sentence_vector = sentence_vector.flatten()  # Flatten to 1D
        if sentence_vector.shape != (embedding_dim,):
            sentence_vector = np.pad(sentence_vector, (0, embedding_dim - len(sentence_vector)), 'constant')
        sentence_vectors.append(sentence_vector)
    return np.array(sentence_vectors)

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG BOOST")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Ada Boost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'

# Load the dataset and GloVe embeddings
df = pd.read_excel(input_file)
embeddings = load_glove_embeddings(glove_file)

# Generate GloVe embeddings for the "answer" column
X = generate_sentence_embeddings(df['answer'], embeddings)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 400000 word vectors.
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 19}
KNN Model Metrics:
Train Accuracy: 0.7017
Train Precision: 0.6764
Train Recall: 0.7017
Train F1 Score: 0.6445
Train Confusion Matrix:
[[  2  11  19]
 [  1  34  97]
 [  0  14 298]]

Test Accuracy: 0.7073
Test Precision: 0.6221
Test Recall: 0.7073
Test F1 Score: 0.6538
Test Confusion Matrix:
[[  0   1  12]
 [  1   7  33]
 [  0  13 138]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      70.17%
Test Accuracy       70.73%
Train-Test Gap      0.56%
ROC-AUC             0.54
Overfitting         No


Running Random Forest model...
Best Params: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 10}
Random Forest Model Metrics:
Train Accuracy: 0.9895
Train Precision: 0.9897
Train Recall: 0.9895
Train

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.8067
Train Precision: 0.8086
Train Recall: 0.8067
Train F1 Score: 0.8032
Train Confusion Matrix:
[[ 15  11   6]
 [  1  91  40]
 [  1  33 278]]

Test Accuracy: 0.7610
Test Precision: 0.7681
Test Recall: 0.7610
Test F1 Score: 0.7619
Test Confusion Matrix:
[[  4   3   6]
 [  2  25  14]
 [  2  22 127]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      80.67%
Test Accuracy       76.10%
Train-Test Gap      4.57%
ROC-AUC             0.73
Overfitting         No


Running XGBoost model...
Best Params: {'subsample': 0.8, 'n_estimators': 100, 'learning_rate': 0.01}
XGBoost Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9836
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 30   0   2]
 [  0 126   6]
 [  0   0 312]]

Test Accuracy: 0.7268
Test Precision: 0.6739
Test Recall: 0.7268
Test F1

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 150, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.8235
Train Precision: 0.8202
Train Recall: 0.8235
Train F1 Score: 0.8193
Train Confusion Matrix:
[[ 27   3   2]
 [  0  81  51]
 [  0  28 284]]

Test Accuracy: 0.7220
Test Precision: 0.7010
Test Recall: 0.7220
Test F1 Score: 0.7082
Test Confusion Matrix:
[[  1   3   9]
 [  1  19  21]
 [  2  21 128]]


Performance Table for Ada Boost:
Metric              Value     
------------------------------
Train Accuracy      82.35%
Test Accuracy       72.20%
Train-Test Gap      10.16%
ROC-AUC             0.64
Overfitting         Yes


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7773
Train Precision: 0.7713
Train Recall: 0.7773
Train F1 Score: 0.7645
Train Confusion Matrix:
[[ 12   8  12]
 [  0  73  59]
 [  3  24 285]]

Test Accuracy: 0.7366
Test Precision: 0.7173
Test Recall: 0.7366
Test F1 Score: 0.7239
Test Confusion Matrix:
[[  3   2   8]
 [  1  17  23]
 [  3  17 131]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      77.73%
Test Accuracy       73.66%
Train-Test Gap      4.07%
ROC-AUC             0.64
Overfitting         No


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.7899
Train Precision: 0.8045
Train Recall: 0.7899
Train F1 Score: 0.7607
Train Confusion Matrix:
[[ 10  10  12]
 [  0  57  75]
 [  0   3 309]]

Test Accuracy: 0.7902
Test Precision: 0.7625
Test Recall: 0.7902
Test F1 Score: 0.7550
Test Confusion Matrix:
[[  2   3   8]
 [  1  14  26]
 [  1   4 146]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      78.99%
Test Accuracy       79.02%
Train-Test Gap      0.03%
ROC-AUC             0.66
Overfitting         No




In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.decomposition import PCA
nltk.download('punkt_tab')

# Mount Google Drive
drive.mount('/content/drive')

# Load GloVe vectors
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    print(f"Loaded {len(embeddings)} word vectors.")
    return embeddings

# Generate sentence embeddings
def generate_sentence_embeddings(sentences, embeddings, embedding_dim=300):
    sentence_vectors = []
    for sentence in sentences:
        tokens = word_tokenize(str(sentence).lower())
        word_vectors = [embeddings[word] for word in tokens if word in embeddings]
        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            sentence_vector = np.zeros(embedding_dim)
        sentence_vector = sentence_vector.flatten()  # Flatten to 1D
        if sentence_vector.shape != (embedding_dim,):
            sentence_vector = np.pad(sentence_vector, (0, embedding_dim - len(sentence_vector)), 'constant')
        sentence_vectors.append(sentence_vector)
    return np.array(sentence_vectors)

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG BOOST")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Ada Boost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
glove_file = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'

# Load the dataset and GloVe embeddings
df = pd.read_excel(input_file)
embeddings = load_glove_embeddings(glove_file)

# Generate GloVe embeddings for the "answer" column
X = generate_sentence_embeddings(df['answer'], embeddings)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 400000 word vectors.
Number of components selected: 43
Running KNN model...
Best Params: {'weights': 'distance', 'n_neighbors': 20}
KNN Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9937
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   1 311]]

Test Accuracy: 0.7073
Test Precision: 0.6240
Test Recall: 0.7073
Test F1 Score: 0.6561
Test Confusion Matrix:
[[  0   1  12]
 [  0   8  33]
 [  0  14 137]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      99.37%
Test Accuracy       70.73%
Train-Test Gap      28.64%
ROC-AUC             0.54
Overfitting         Yes


Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precisio

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.8824
Train Precision: 0.8818
Train Recall: 0.8824
Train F1 Score: 0.8765
Train Confusion Matrix:
[[ 18   9   5]
 [  1  97  34]
 [  1   6 305]]

Test Accuracy: 0.7610
Test Precision: 0.7366
Test Recall: 0.7610
Test F1 Score: 0.7431
Test Confusion Matrix:
[[  1   4   8]
 [  1  21  19]
 [  1  16 134]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      88.24%
Test Accuracy       76.10%
Train-Test Gap      12.14%
ROC-AUC             0.68
Overfitting         Yes


Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7561
Test Precision: 0.7582
Test Recall: 0.7561
Test F1

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7059
Train Precision: 0.6965
Train Recall: 0.7059
Train F1 Score: 0.6816
Train Confusion Matrix:
[[  7  12  13]
 [  1  52  79]
 [  0  35 277]]

Test Accuracy: 0.6780
Test Precision: 0.7017
Test Recall: 0.6780
Test F1 Score: 0.6643
Test Confusion Matrix:
[[  1   1  11]
 [  0  16  25]
 [  0  29 122]]


Performance Table for Ada Boost:
Metric              Value     
------------------------------
Train Accuracy      70.59%
Test Accuracy       67.80%
Train-Test Gap      2.78%
ROC-AUC             0.58
Overfitting         No


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7290
Train Precision: 0.7108
Train Recall: 0.7290
Train F1 Score: 0.7031
Train Confusion Matrix:
[[  9   8  15]
 [  2  52  78]
 [  4  22 286]]

Test Accuracy: 0.7463
Test Precision: 0.7097
Test Recall: 0.7463
Test F1 Score: 0.7189
Test Confusion Matrix:
[[  2   1  10]
 [  3  13  25]
 [  2  11 138]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      72.90%
Test Accuracy       74.63%
Train-Test Gap      1.73%
ROC-AUC             0.63
Overfitting         No


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9076
Train Precision: 0.9120
Train Recall: 0.9076
Train F1 Score: 0.9034
Train Confusion Matrix:
[[ 27   4   1]
 [  0  95  37]
 [  0   2 310]]

Test Accuracy: 0.7171
Test Precision: 0.6895
Test Recall: 0.7171
Test F1 Score: 0.7015
Test Confusion Matrix:
[[  1   3   9]
 [  3  16  22]
 [  2  19 130]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      90.76%
Test Accuracy       71.71%
Train-Test Gap      19.05%
ROC-AUC             0.63
Overfitting         Yes




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_text import LimeTextExplainer
from google.colab import drive
import numpy as np

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Create an embedding matrix for the dataset
def create_embedding_matrix(data, embeddings, text_column='TEXT', embedding_dim=100):
    matrix = []
    for text in data[text_column]:
        words = text.split()
        # Get the word vectors, default to zero vector if the word is not found
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]

        # If there are no valid word vectors, append a zero vector
        if len(word_vectors) == 0:
            vec = np.zeros(embedding_dim)
        else:
            vec = np.mean(word_vectors, axis=0)
        matrix.append(vec)
    return np.array(matrix)

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    # Map labels to "not strong" and "strong"
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)

    # Use only the ANSWER column for input features
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')

    return data

# Train KNN classifier with manual hyperparameters
def train_knn_manual(X_train, y_train, n_neighbors=19, weights='uniform', metric='manhattan'):
    # Define the KNN model with manual parameters
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)
    knn.fit(X_train, y_train)
    return knn

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        # Predict probabilities for the input texts
        embeddings = create_embedding_matrix(pd.DataFrame(texts, columns=['TEXT']), glove_embeddings)
        return model.predict_proba(embeddings)

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
    glove_file_path = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'  # Update with your GloVe file path

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load GloVe embeddings
    global glove_embeddings
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create embedding matrix
    X = create_embedding_matrix(data, glove_embeddings)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train KNN classifier with manual hyperparameters
    knn_model = train_knn_manual(X_train, y_train, n_neighbors=3, weights='distance', metric='manhattan')

    # Predict and evaluate
    y_pred = knn_model.predict(X_test)

    # Overall Metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_f1_score = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print()
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print(f"Precision: {weighted_precision:.2f}")
    print(f"Recall: {weighted_recall:.2f}")
    print(f"F1 Score: {weighted_f1_score:.2f}")

    # Built-in confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 237 # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(knn_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    # LIME explanation for a sample text
    sample_index = 6  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(knn_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier  # Import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_text import LimeTextExplainer
from google.colab import drive
import numpy as np

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Create an embedding matrix for the dataset
def create_embedding_matrix(data, embeddings, text_column='TEXT', embedding_dim=100):
    matrix = []
    for text in data[text_column]:
        words = text.split()
        # Get the word vectors, default to zero vector if the word is not found
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]

        # If there are no valid word vectors, append a zero vector
        if len(word_vectors) == 0:
            vec = np.zeros(embedding_dim)
        else:
            vec = np.mean(word_vectors, axis=0)
        matrix.append(vec)
    return np.array(matrix)

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    # Map labels to "not strong" and "strong"
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)

    # Use only the ANSWER column for input features
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')

    return data

# Train MLP classifier with manual hyperparameters
def train_mlp_manual(X_train, y_train, hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=200):
    # Define the MLP model with manual parameters
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, max_iter=max_iter)
    mlp.fit(X_train, y_train)
    return mlp

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        # Predict probabilities for the input texts
        embeddings = create_embedding_matrix(pd.DataFrame(texts, columns=['TEXT']), glove_embeddings)
        return model.predict_proba(embeddings)

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
    glove_file_path = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'  # Update with your GloVe file path

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load GloVe embeddings
    global glove_embeddings
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create embedding matrix
    X = create_embedding_matrix(data, glove_embeddings)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train MLP classifier with manual hyperparameters
    mlp_model = train_mlp_manual(X_train, y_train, hidden_layer_sizes=(100,), activation='tanh', solver='adam', max_iter=300)

    # Predict and evaluate
    y_pred = mlp_model.predict(X_test)

    # Overall Metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_f1_score = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print()
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print(f"Precision: {weighted_precision:.2f}")
    print(f"Recall: {weighted_recall:.2f}")
    print(f"F1 Score: {weighted_f1_score:.2f}")

    # Built-in confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 237 # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    # LIME explanation for another sample text
    sample_index = 6  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_text import LimeTextExplainer
from google.colab import drive
import numpy as np

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Create an embedding matrix for the dataset
def create_embedding_matrix(data, embeddings, text_column='TEXT', embedding_dim=100):
    matrix = []
    for text in data[text_column]:
        words = text.split()
        # Get the word vectors, default to zero vector if the word is not found
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]

        # If there are no valid word vectors, append a zero vector
        if len(word_vectors) == 0:
            vec = np.zeros(embedding_dim)
        else:
            vec = np.mean(word_vectors, axis=0)
        matrix.append(vec)
    return np.array(matrix)

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    # Map labels to "not strong" and "strong"
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)

    # Use only the ANSWER column for input features
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')

    return data

# Train Logistic Regression classifier
def train_logistic_regression(X_train, y_train, penalty='l2', C=10, solver='lbfgs'):
    # Define Logistic Regression model with penalty and C
    lr = LogisticRegression(solver=solver, max_iter=1000, penalty=penalty, C=C)
    lr.fit(X_train, y_train)
    return lr

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        # Predict probabilities for the input texts
        embeddings = create_embedding_matrix(pd.DataFrame(texts, columns=['TEXT']), glove_embeddings)
        return model.predict_proba(embeddings)

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
    glove_file_path = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'  # Update with your GloVe file path

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load GloVe embeddings
    global glove_embeddings
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create embedding matrix
    X = create_embedding_matrix(data, glove_embeddings)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train Logistic Regression classifier with penalty and C
    penalty = 'l2'  # Regularization type (l2 for Ridge regularization, l1 for Lasso regularization)
    C = 10  # Regularization strength (lower values mean stronger regularization)
    lr_model = train_logistic_regression(X_train, y_train, penalty=penalty, C=C)

    # Predict and evaluate
    y_pred = lr_model.predict(X_test)

    # Overall Metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_f1_score = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print()
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print(f"Precision: {weighted_precision:.2f}")
    print(f"Recall: {weighted_recall:.2f}")
    print(f"F1 Score: {weighted_f1_score:.2f}")

    # Built-in confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 237  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(lr_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    # LIME explanation for another sample text
    sample_index = 6  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(lr_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
from lime.lime_text import LimeTextExplainer
from google.colab import drive

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Create an embedding matrix for the dataset
def create_embedding_matrix(data, embeddings, text_column='TEXT', embedding_dim=100):
    matrix = []
    for text in data[text_column]:
        words = text.split()
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]
        if len(word_vectors) == 0:
            vec = np.zeros(embedding_dim)
        else:
            vec = np.mean(word_vectors, axis=0)
        matrix.append(vec)
    return np.array(matrix)

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')
    return data

# Train SVM classifier with kernel and C as hyperparameters
def train_svm(X_train, y_train, kernel='rbf', C=10):
    # Initialize SVC with probability=True to enable predict_proba
    svm = SVC(kernel=kernel, C=C, probability=True)
    svm.fit(X_train, y_train)
    return svm

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        embeddings = create_embedding_matrix(pd.DataFrame(texts, columns=['TEXT']), glove_embeddings)
        return model.predict_proba(embeddings)

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
    glove_file_path = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'  # Update with your GloVe file path

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load GloVe embeddings
    global glove_embeddings
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create embedding matrix
    X = create_embedding_matrix(data, glove_embeddings)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train SVM classifier with kernel and C parameters
    kernel = 'rbf'  # Kernel type (can be 'linear', 'rbf', 'poly', etc.)
    C = 10  # Regularization strength
    svm_model = train_svm(X_train, y_train, kernel=kernel, C=C)

    # Predict and evaluate
    y_pred = svm_model.predict(X_test)

    # Evaluation metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print("Classification Report:\n", classification_report(y_test, y_pred))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 237  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(svm_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Accuracy Score: 0.71
Classification Report:
               precision    recall  f1-score   support

  not strong       0.50      0.25      0.33        40
      strong       0.74      0.90      0.81        97

    accuracy                           0.71       137
   macro avg       0.62      0.57      0.57       137
weighted avg       0.67      0.71      0.67       137

Confusion Matrix:
 [[10 30]
 [10 87]]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_text import LimeTextExplainer
from google.colab import drive
import numpy as np

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Create an embedding matrix for the dataset
def create_embedding_matrix(data, embeddings, text_column='TEXT', embedding_dim=100):
    matrix = []
    for text in data[text_column]:
        words = text.split()
        # Get the word vectors, default to zero vector if the word is not found
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]

        # If there are no valid word vectors, append a zero vector
        if len(word_vectors) == 0:
            vec = np.zeros(embedding_dim)
        else:
            vec = np.mean(word_vectors, axis=0)
        matrix.append(vec)
    return np.array(matrix)

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    # Map labels to "not strong" and "strong"
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)

    # Use only the ANSWER column for input features
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')

    return data

# Train AdaBoost classifier
def train_adaboost(X_train, y_train, n_estimators=150, learning_rate=1.0):
    # Define AdaBoostClassifier model
    adaboost = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate)
    adaboost.fit(X_train, y_train)
    return adaboost

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        # Predict probabilities for the input texts
        embeddings = create_embedding_matrix(pd.DataFrame(texts, columns=['TEXT']), glove_embeddings)
        return model.predict_proba(embeddings)

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
    glove_file_path = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'  # Update with your GloVe file path

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load GloVe embeddings
    global glove_embeddings
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create embedding matrix
    X = create_embedding_matrix(data, glove_embeddings)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train AdaBoost classifier
    adaboost_model = train_adaboost(X_train, y_train)

    # Predict and evaluate
    y_pred = adaboost_model.predict(X_test)

    # Overall Metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_f1_score = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print()
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print(f"Precision: {weighted_precision:.2f}")
    print(f"Recall: {weighted_recall:.2f}")
    print(f"F1 Score: {weighted_f1_score:.2f}")

    # Built-in confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 237  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(adaboost_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    # LIME explanation for another sample text
    sample_index = 6  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(adaboost_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier  # Import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_text import LimeTextExplainer
from google.colab import drive
import numpy as np

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Create an embedding matrix for the dataset
def create_embedding_matrix(data, embeddings, text_column='TEXT', embedding_dim=100):
    matrix = []
    for text in data[text_column]:
        words = text.split()
        # Get the word vectors, default to zero vector if the word is not found
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]

        # If there are no valid word vectors, append a zero vector
        if len(word_vectors) == 0:
            vec = np.zeros(embedding_dim)
        else:
            vec = np.mean(word_vectors, axis=0)
        matrix.append(vec)
    return np.array(matrix)

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    # Map labels to "not strong" and "strong"
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)

    # Use only the ANSWER column for input features
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')

    return data

# Train MLP classifier with manual hyperparameters
def train_mlp_manual(X_train, y_train, hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=200):
    # Define the MLP model with manual parameters
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, max_iter=max_iter)
    mlp.fit(X_train, y_train)
    return mlp

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        # Predict probabilities for the input texts
        embeddings = create_embedding_matrix(pd.DataFrame(texts, columns=['TEXT']), glove_embeddings)
        return model.predict_proba(embeddings)

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
    glove_file_path = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'  # Update with your GloVe file path

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load GloVe embeddings
    global glove_embeddings
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create embedding matrix
    X = create_embedding_matrix(data, glove_embeddings)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train MLP classifier with manual hyperparameters
    mlp_model = train_mlp_manual(X_train, y_train, hidden_layer_sizes=(100,), activation='tanh', solver='adam', max_iter=300)

    # Predict and evaluate
    y_pred = mlp_model.predict(X_test)

    # Overall Metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_f1_score = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print()
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print(f"Precision: {weighted_precision:.2f}")
    print(f"Recall: {weighted_recall:.2f}")
    print(f"F1 Score: {weighted_f1_score:.2f}")

    # Built-in confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 673 # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    # LIME explanation for another sample text
    sample_index = 6  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


Output hidden; open in https://colab.research.google.com to view.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier  # Import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_text import LimeTextExplainer
from google.colab import drive
import numpy as np

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Create an embedding matrix for the dataset
def create_embedding_matrix(data, embeddings, text_column='TEXT', embedding_dim=100):
    matrix = []
    for text in data[text_column]:
        words = text.split()
        # Get the word vectors, default to zero vector if the word is not found
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]

        # If there are no valid word vectors, append a zero vector
        if len(word_vectors) == 0:
            vec = np.zeros(embedding_dim)
        else:
            vec = np.mean(word_vectors, axis=0)
        matrix.append(vec)
    return np.array(matrix)

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    # Map labels to "not strong" and "strong"
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)

    # Use only the ANSWER column for input features
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')

    return data

# Train MLP classifier with manual hyperparameters
def train_mlp_manual(X_train, y_train, hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=200):
    # Define the MLP model with manual parameters
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, max_iter=max_iter)
    mlp.fit(X_train, y_train)
    return mlp

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        # Predict probabilities for the input texts
        embeddings = create_embedding_matrix(pd.DataFrame(texts, columns=['TEXT']), glove_embeddings)
        return model.predict_proba(embeddings)

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
    glove_file_path = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'  # Update with your GloVe file path

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load GloVe embeddings
    global glove_embeddings
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create embedding matrix
    X = create_embedding_matrix(data, glove_embeddings)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train MLP classifier with manual hyperparameters
    mlp_model = train_mlp_manual(X_train, y_train, hidden_layer_sizes=(100,), activation='tanh', solver='adam', max_iter=300)

    # Predict and evaluate
    y_pred = mlp_model.predict(X_test)

    # Overall Metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_f1_score = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print()
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print(f"Precision: {weighted_precision:.2f}")
    print(f"Recall: {weighted_recall:.2f}")
    print(f"F1 Score: {weighted_f1_score:.2f}")

    # Built-in confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 673 # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    # LIME explanation for another sample text
    sample_index = 532  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    sample_index = 432  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    sample_index = 237  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    sample_index = 321  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


Output hidden; open in https://colab.research.google.com to view.

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier  # Import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_text import LimeTextExplainer
from google.colab import drive
import numpy as np

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Create an embedding matrix for the dataset
def create_embedding_matrix(data, embeddings, text_column='TEXT', embedding_dim=100):
    matrix = []
    for text in data[text_column]:
        words = text.split()
        # Get the word vectors, default to zero vector if the word is not found
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]

        # If there are no valid word vectors, append a zero vector
        if len(word_vectors) == 0:
            vec = np.zeros(embedding_dim)
        else:
            vec = np.mean(word_vectors, axis=0)
        matrix.append(vec)
    return np.array(matrix)

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    # Map labels to "not strong" and "strong"
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)

    # Use only the ANSWER column for input features
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')

    return data

# Train MLP classifier with manual hyperparameters
def train_mlp_manual(X_train, y_train, hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=200):
    # Define the MLP model with manual parameters
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, max_iter=max_iter)
    mlp.fit(X_train, y_train)
    return mlp

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        # Predict probabilities for the input texts
        embeddings = create_embedding_matrix(pd.DataFrame(texts, columns=['TEXT']), glove_embeddings)
        return model.predict_proba(embeddings)

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
    glove_file_path = '/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt'  # Update with your GloVe file path

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load GloVe embeddings
    global glove_embeddings
    glove_embeddings = load_glove_embeddings(glove_file_path)

    # Create embedding matrix
    X = create_embedding_matrix(data, glove_embeddings)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train MLP classifier with manual hyperparameters
    mlp_model = train_mlp_manual(X_train, y_train, hidden_layer_sizes=(100,), activation='tanh', solver='adam', max_iter=300)

    # Predict and evaluate
    y_pred = mlp_model.predict(X_test)

    # Overall Metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_f1_score = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print()
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print(f"Precision: {weighted_precision:.2f}")
    print(f"Recall: {weighted_recall:.2f}")
    print(f"F1 Score: {weighted_f1_score:.2f}")

    # Built-in confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 32 # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    # LIME explanation for another sample text
    sample_index = 56 # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    sample_index = 12  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    sample_index = 78  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

    sample_index = 278  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(mlp_model, sample_text, label_classes=['not strong', 'strong'])
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


Output hidden; open in https://colab.research.google.com to view.